## iii-org rancher pipeline data generator example

透過服務函式建立完整rancher pipeline資料內容(教學範例)

#### 參考
* [使用-python-讀取-yaml-檔案-d3f413d7dd6](https://medium.com/bryanyang0528/%E4%BD%BF%E7%94%A8-python-%E8%AE%80%E5%8F%96-yaml-%E6%AA%94%E6%A1%88-d3f413d7dd6)
* [yaml.readthedocs](https://yaml.readthedocs.io/en/latest/example.html)

In [5]:
import sys
from ruamel.yaml import YAML

In [6]:
inp = """
stages:
  # 建立使用者專用的 Docker Image
  - name: Publish User Web image
    steps:
    - publishImageConfig:
        dockerfilePath: ./Dockerfile
        buildContext: .
        tag: app/${CICD_GIT_REPO_NAME}-web:latest
        pushRemote: true
        registry: 10.50.1.63:5443
      env:
        PLUGIN_INSECURE: "true"

  # 部屬使用者應用(app)
  - name: Deploy app-web
    steps:
    - applyAppConfig:
        catalogTemplate: cattle-global-data:iii-dev-charts3-web
        version: 0.1.0
        name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
        targetNamespace: p-v2nzs-pipeline
        answers:
          # 基礎pipeline參數
          git.url: ${CICD_GIT_URL}
          git.branch: ${CICD_GIT_BRANCH}
          git.commitID: ${CICD_GIT_COMMIT}
          git.repoName: ${CICD_GIT_REPO_NAME}
          # web參數設定
          web.type: uwsgi-flask
          web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
          web.tag: latest
          checkmarx.enabled: false
          newman.enabled: false
"""

In [7]:
yaml = YAML()
code = yaml.load(inp)

In [8]:
yaml.dump(code, sys.stdout)

stages:
  # 建立flask專用的 Docker Image
- name: Publish Flask image
  steps:
  - publishImageConfig:
      dockerfilePath: ./Dockerfile
      buildContext: .
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
      pushRemote: true
      registry: 10.50.1.63:5443
    env:
      PLUGIN_INSECURE: 'true'

  # 部屬使用者應用(app)
- name: Deploy app-web
  steps:
  - applyAppConfig:
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      version: 0.1.0
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: p-v2nzs-pipeline
      answers:
          # 基礎pipeline參數
        git.url: ${CICD_GIT_URL}
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
          # web參數設定
        web.type: uwsgi-flask
        web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
        web.tag: latest
        checkmarx.enabled: false
        newman.enabled: false


In [10]:
tcode = code['stages'].pop(1)

In [11]:
yaml.dump(tcode, sys.stdout)

name: Deploy app-web
steps:
- applyAppConfig:
    catalogTemplate: cattle-global-data:iii-dev-charts3-web
    version: 0.1.0
    name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
    targetNamespace: p-v2nzs-pipeline
    answers:
          # 基礎pipeline參數
      git.url: ${CICD_GIT_URL}
      git.branch: ${CICD_GIT_BRANCH}
      git.commitID: ${CICD_GIT_COMMIT}
      git.repoName: ${CICD_GIT_REPO_NAME}
          # web參數設定
      web.type: uwsgi-flask
      web.image: 10.50.1.63:5443/app/${CICD_GIT_REPO_NAME}-web
      web.tag: latest
      checkmarx.enabled: false
      newman.enabled: false


In [15]:
with open('./stander-rancher-pipeline.yaml', 'r') as stream:
    data = yaml.load(stream)

In [17]:
yaml.dump(data, sys.stdout)

stages:
  # 建立使用者專用的 Docker Image
- name: Publish User Web image
  steps:
  - publishImageConfig:
      dockerfilePath: ./Dockerfile
      buildContext: .
        # (疑問: 鏡像上傳名稱與TAG)
      tag: app/${CICD_GIT_REPO_NAME}-web:latest
      pushRemote: true
        # (開發環境: registry IP)
      registry: 10.50.1.63:5443
        # (開發環境: registry是否有有效憑證)
    env:
      PLUGIN_INSECURE: 'true'

  # 部屬使用者應用(app)
- name: Deploy app-web
  steps:
  - applyAppConfig:
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      version: 0.1.0
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
        # (開發環境: 要部屬的namespace名稱)
      targetNamespace: p-v2nzs-pipeline
      answers:
          # (固定: 基礎pipeline參數)
        git.url: ${CICD_GIT_URL}
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
          # (使用者選擇: 專案部屬型態web參數設定)
        web.type: uwsgi-flask
          # (開發環境: 前一步上傳鏡像所需的參數)
        web.image: 10.50.1.